In [7]:
import feature_engineering as fe
import train_utils as tu
from persistent_cache import memo, PersistentDict as Perd
from sklearn.cross_validation import StratifiedKFold, KFold
import numpy as np

In [24]:
X_train, X_test = fe.oh_med_cut()
y = fe.get_y()

In [35]:
def corr(a, b):
    return np.corrcoef(a, b)[0][1]
#def sqr_corr(X_train, X_test, y, threshold):

threshold = 0.12
corrs = []
for i in range(X_train.shape[1]):
    corrs.append(corr(X_train[:, i], y))
gooduns = [i for i, c in enumerate(corrs) if abs(c) > threshold]
rest = [i for i, c in enumerate(corrs) if abs(c) <= threshold]

tot = np.vstack([X_train, X_test])
good_feats = tot[:, gooduns]
print good_feats.shape
meh_feats = tot[:, rest]
poly_feats = PolynomialFeatures(2).fit_transform(good_feats)
assert set(poly_feats[:, 0]) == {1}
new_tot = np.hstack([poly_feats, meh_feats[:, 1:]])

(79146, 30)


In [33]:
def add_poly_feats(X_train, X_test, y, threshold):
    corrs = []
    for i in range(X_train.shape[1]):
        corrs.append(corr(X_train[:, i], y))
    gooduns = [i for i, c in enumerate(corrs) if abs(c) > threshold]
    rest = [i for i, c in enumerate(corrs) if abs(c) <= threshold]
    tot = np.vstack([X_train, X_test])
    good_feats = tot[:, gooduns]
    meh_feats = tot[:, rest]
    poly_feats = PolynomialFeatures(2).fit_transform(good_feats)
    assert set(poly_feats[:, 0]) == {1}
    new_tot = np.hstack([poly_feats, meh_feats[:, 1:]])
    n = len(y)
    return new_tot[:n, :], new_tot[n:, :]

((79146, 484), (79146, 331))

In [36]:
new_tot.shape, tot.shape

((79146, 796), (79146, 331))

In [13]:
poly = PolynomialFeatures(2)

In [15]:
poly.fit_transform(np.array([[7,1],[2,3],[4,5]]))

array([[ 1,  7,  1, 49,  7,  1],
       [ 1,  2,  3,  4,  6,  9],
       [ 1,  4,  5, 16, 20, 25]])

In [22]:
x, xx = fe.nan_count()

In [23]:
corr(x.ravel(), y)

0.098648219879977239